In [1]:
import sys
import os
from pathlib import Path
sys.path.append(Path.cwd().parent.as_posix())
from util import *
from visualizer import visualizer
vis = visualizer()

In [2]:
structures = test_structures(100)

  0%|          | 0/100 [00:00<?, ?it/s]

[I] Loaded 100 structures


In [3]:
cif, sym_info, num_sites, formula = process_prim_with_sym_infos(structures[0])

In [6]:
sym_info

{'anchors': [0, 1, 2, 2],
 'wyckoff_ops': [[[0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 1.0]],
  [[0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.5],
   [0.0, 0.0, 0.0, 0.5],
   [0.0, 0.0, 0.0, 1.0]],
  [[0.0, 0.0, 0.0, 0.5],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 0.5],
   [0.0, 0.0, 0.0, 1.0]],
  [[0.0, 0.0, 0.0, 0.5],
   [0.0, 0.0, 0.0, 0.5],
   [0.0, 0.0, 0.0, 0.0],
   [0.0, 0.0, 0.0, 1.0]],
  [[0.0, 0.0, 0.0, 0.5],
   [0.0, 0.0, 0.0, 0.5],
   [0.0, 0.0, 0.0, 0.5],
   [0.0, 0.0, 0.0, 1.0]],
  [[0.0, 0.0, 0.0, 0.5],
   [0.0, 0.0, 0.0, 1.0],
   [0.0, 0.0, 0.0, 1.0],
   [0.0, 0.0, 0.0, 1.0]],
  [[0.0, 0.0, 0.0, 1.0],
   [0.0, 0.0, 0.0, 0.5],
   [0.0, 0.0, 0.0, 1.0],
   [0.0, 0.0, 0.0, 1.0]],
  [[0.0, 0.0, 0.0, 1.0],
   [0.0, 0.0, 0.0, 1.0],
   [0.0, 0.0, 0.0, 0.5],
   [0.0, 0.0, 0.0, 1.0]],
  [[0.0, 0.0, 0.0, 0.25],
   [0.0, 0.0, 0.0, 0.25],
   [0.0, 0.0, 0.0, 0.25],
   [0.0, 0.0, 0.0, 1.0]],
  [[0.0, 0.0, 0.0, 0.25],
   [0.0, 0.0, 0.0, 0.

In [7]:
space_group = sym_info['spacegroup']

In [ ]:
prim_plat, conv_plat, perm, rank = project_random_lattice_to_symmetry_prim_conv_permute(
    initialize_random_lattice().matrix, sym_info["conv_to_prim"], sym_info["spacegroup"]
)

In [10]:
conv_psites, conv_species = project_sites_prim_with_species(
    initialize_random_atoms(num_sites), sym_info
)

In [30]:
conv_plat

array([[9.81137753, 0.        , 0.        ],
       [0.        , 9.81137753, 0.        ],
       [0.        , 0.        , 9.81137753]])

In [11]:
perm_conv_psite = np.dot(perm, conv_psites.T).T
perm_conv_plat = perm_for_A2[space_group].dot(conv_plat).dot(perm_for_A2[space_group].T)
# print(prim_plat)
prim_species, prim_psites = map_sites_conv_to_prim_lat(
    perm_conv_psite, conv_species, perm_conv_plat, prim_plat
)

In [13]:
perm_conv_psite

array([[0.  , 0.  , 0.  ],
       [0.  , 0.5 , 0.5 ],
       [0.5 , 0.  , 0.5 ],
       [0.5 , 0.5 , 0.  ],
       [0.5 , 0.5 , 0.5 ],
       [0.5 , 0.  , 0.  ],
       [0.  , 0.5 , 0.  ],
       [0.  , 0.  , 0.5 ],
       [0.25, 0.25, 0.25],
       [0.25, 0.25, 0.75],
       [0.25, 0.75, 0.75],
       [0.25, 0.75, 0.25],
       [0.75, 0.25, 0.75],
       [0.75, 0.25, 0.25],
       [0.75, 0.75, 0.25],
       [0.75, 0.75, 0.75]])

In [12]:
prim_psites

[array([0., 0., 0.]),
 array([0.5, 0.5, 0.5]),
 array([0.25, 0.25, 0.25]),
 array([0.75, 0.75, 0.75])]

In [ ]:
def map_sites_conv_to_prim_lat_triu(conv_sites, conv_species, conv_lat, prim_lat):
    prim_lattice_inv = np.linalg.inv(prim_lat)
    new_fracs = []
    new_species = []
    conv_sites = np.dot(conv_sites, conv_lat)
    new_frac = np.dot(conv_sites, prim_lattice_inv)
    for cart_coords, specie in zip(conv_sites, conv_species):
        new_frac = np.dot(cart_coords, prim_lattice_inv)
        # if not any(partial(pbc_diff,new_frac) == partial(f) for f in new_fracs):
        #     new_species.append(site.specie)
        if not any(map(partial(is_periodic_image, new_frac), new_fracs)):
            new_fracs.append(new_frac % 1)
            new_species.append(specie)
        new_fracs = [new_frac % 1 for new_frac in new_fracs]

    # Rhombohedral is not implemented yet
    return new_species, new_fracs

In [15]:
prim_lattice_inv =  np.linalg.inv(prim_plat)
new_fracs = []
new_species = []
conv_sites = np.dot(perm_conv_psite, conv_plat)

In [16]:
conv_sites

array([[0.        , 0.        , 0.        ],
       [0.        , 4.90568876, 4.90568876],
       [4.90568876, 0.        , 4.90568876],
       [4.90568876, 4.90568876, 0.        ],
       [4.90568876, 4.90568876, 4.90568876],
       [4.90568876, 0.        , 0.        ],
       [0.        , 4.90568876, 0.        ],
       [0.        , 0.        , 4.90568876],
       [2.45284438, 2.45284438, 2.45284438],
       [2.45284438, 2.45284438, 7.35853314],
       [2.45284438, 7.35853314, 7.35853314],
       [2.45284438, 7.35853314, 2.45284438],
       [7.35853314, 2.45284438, 7.35853314],
       [7.35853314, 2.45284438, 2.45284438],
       [7.35853314, 7.35853314, 2.45284438],
       [7.35853314, 7.35853314, 7.35853314]])

In [17]:
new_frac = np.dot(conv_sites, prim_lattice_inv)

In [26]:
(new_frac % 1)

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.65704574e-17, 1.00000000e+00],
       [2.65704574e-17, 0.00000000e+00, 1.00000000e+00],
       [2.65704574e-17, 1.00000000e+00, 0.00000000e+00],
       [5.00000000e-01, 5.00000000e-01, 5.00000000e-01],
       [5.00000000e-01, 5.00000000e-01, 5.00000000e-01],
       [5.00000000e-01, 5.00000000e-01, 5.00000000e-01],
       [5.00000000e-01, 5.00000000e-01, 5.00000000e-01],
       [2.50000000e-01, 2.50000000e-01, 2.50000000e-01],
       [7.50000000e-01, 7.50000000e-01, 7.50000000e-01],
       [2.50000000e-01, 2.50000000e-01, 2.50000000e-01],
       [7.50000000e-01, 7.50000000e-01, 7.50000000e-01],
       [2.50000000e-01, 2.50000000e-01, 2.50000000e-01],
       [7.50000000e-01, 7.50000000e-01, 7.50000000e-01],
       [2.50000000e-01, 2.50000000e-01, 2.50000000e-01],
       [7.50000000e-01, 7.50000000e-01, 7.50000000e-01]])

In [29]:
(new_frac % 1.0) % 1.0

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.65704574e-17, 0.00000000e+00],
       [2.65704574e-17, 0.00000000e+00, 0.00000000e+00],
       [2.65704574e-17, 0.00000000e+00, 0.00000000e+00],
       [5.00000000e-01, 5.00000000e-01, 5.00000000e-01],
       [5.00000000e-01, 5.00000000e-01, 5.00000000e-01],
       [5.00000000e-01, 5.00000000e-01, 5.00000000e-01],
       [5.00000000e-01, 5.00000000e-01, 5.00000000e-01],
       [2.50000000e-01, 2.50000000e-01, 2.50000000e-01],
       [7.50000000e-01, 7.50000000e-01, 7.50000000e-01],
       [2.50000000e-01, 2.50000000e-01, 2.50000000e-01],
       [7.50000000e-01, 7.50000000e-01, 7.50000000e-01],
       [2.50000000e-01, 2.50000000e-01, 2.50000000e-01],
       [7.50000000e-01, 7.50000000e-01, 7.50000000e-01],
       [2.50000000e-01, 2.50000000e-01, 2.50000000e-01],
       [7.50000000e-01, 7.50000000e-01, 7.50000000e-01]])

In [7]:
structures[0].sites[1].coords

array([-6.66133815e-16,  6.18349240e+00,  6.66133815e-16])

In [17]:
np.dot(structures[0].frac_coords,structures[0].lattice.matrix)

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-6.47243652e-16,  6.18349240e+00,  7.24353932e-16],
       [ 1.20467384e-16,  9.27523859e+00,  6.66133815e-16],
       [-2.03154442e-16,  3.09174620e+00, -3.38484840e-16]])

In [3]:
lat_perm = np.zeros((231, 3, 3, 3, 3, 3))
lat_perm[:,:,:,:, 0, 0] = 1
lat_perm[:,:,:,:, 1, 1] = 1
lat_perm[:,:,:,:, 2, 2] = 1
# 1, 0, 2 implies lat(1) > lat(0) > lat(2)
possible_perms = [[0, 1, 2], [0, 2, 1], [1, 0, 2], [1, 2, 0], [2, 0, 1], [2, 1, 0]]

# orthorhombic with A:
# for i in [38, 39, 40, 41]:
#     lat_perm[i, 0, 1, 2] = lat_perm[0, 0, 1, 2].dot(lat_perm[0, 1, 2, 0])
#     lat_perm[i, 0, 2, 1] = lat_perm[0, 0, 2, 1].dot(lat_perm[0, 1, 2, 0])
#     lat_perm[i, 1, 0, 2] = lat_perm[0, 1, 0, 2].dot(lat_perm[0, 1, 2, 0])
#     lat_perm[i, 1, 2, 0] = lat_perm[0, 1, 2, 0].dot(lat_perm[0, 1, 2, 0])
#     lat_perm[i, 2, 0, 1] = lat_perm[0, 2, 0, 1].dot(lat_perm[0, 1, 2, 0])
#     lat_perm[i, 2, 1, 0] = lat_perm[0, 2, 1, 0].dot(lat_perm[0, 1, 2, 0])

perm_for_A1 = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
# perm_a = np.array([[1, 0, 0], [0, 0, 1], [0, 1, 0]])
perm_for_A2 = np.eye(3).reshape(1,3,3).repeat(231, axis=0)
# perm_for_A2 = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])

for i in [38, 39, 40, 41]:
    lat_perm[i, 0, 1, 2] = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])
    lat_perm[i, 0, 2, 1] = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
    lat_perm[i, 1, 0, 2] = np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]])
    lat_perm[i, 1, 2, 0] = np.array([[1, 0, 0], [0, 0, 1], [0, 1, 0]])
    lat_perm[i, 2, 0, 1] = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 1]])
    lat_perm[i, 2, 1, 0] = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

    for p in possible_perms:
        lat_perm[i, p[0], p[1], p[2]] = perm_for_A1 @ lat_perm[0, p[0], p[1], p[2]]
    perm_for_A2[i] = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])
    # lat_perm[i, 0, 1, 2] = perm_for_A1 @ lat_perm[0, 0, 1, 2]
    # lat_perm[i, 0, 2, 1] = perm_for_A1 @ lat_perm[0, 0, 2, 1]
    # lat_perm[i, 1, 0, 2] = perm_for_A1 @ lat_perm[0, 1, 0, 2]
    # lat_perm[i, 1, 2, 0] = perm_for_A1 @ lat_perm[0, 1, 2, 0]
    # lat_perm[i, 2, 0, 1] = perm_for_A1 @ lat_perm[0, 2, 0, 1]
    # lat_perm[i, 2, 1, 0] = perm_for_A1 @ lat_perm[0, 2, 1, 0]
print (lat_perm[41,0,1,2])
print (lat_perm[0,0,1,2])

[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [23]:
for p in possible_perms:
    print(lat_perm[0, p[0], p[1], p[2]])

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [6]:
# def map_sites_conv_to_prim2(conv, prim):
#     prim_lattice_inv = prim.lattice.inv_matrix
#     new_fracs = []
#     new_species = []
#     for site in conv:
#         new_frac = np.dot(site.coords, prim_lattice_inv)
#         # if not any(partial(pbc_diff,new_frac) == partial(f) for f in new_fracs):
#         #     new_species.append(site.specie)
#         if not any(map(partial(is_periodic_image, new_frac), new_fracs)):
#             new_fracs.append(new_frac % 1)
#             new_species.append(site.specie)
#         new_fracs = [new_frac % 1 for new_frac in new_fracs]

#     # Rhombohedral is not implemented yet
#     return new_species, new_fracs


In [10]:
res = p_map(test_space_group_projection_prim3, structures, num_cpus=16)

  0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
sgn_correct = []
sgn_failed = []
len_correct = []
len_failed = []
spga_len_correct = []
spga_len_failed = []
errors = []

for i in range(len(res)):
    if res[i]["correct"]:
        sgn_correct.append(i)
    else:
        sgn_failed.append(i)
    if res[i]["correct_species_len"]:
        len_correct.append(i)
    else:
        len_failed.append(i)
    if res[i]["correct_species_len_spga"]:
        spga_len_correct.append(i)
    else:
        spga_len_failed.append(i)
    if res[i]["error"]:
        errors.append(i)

print(len(sgn_correct), len(len_correct), len(errors))
print(len(sgn_correct)/len(res), len(len_correct)/len(res), len(errors)/len(res))

100 100 0
1.0 1.0 0.0


In [12]:
structures[46]

Structure Summary
Lattice
    abc : 4.936342139999999 4.936342139999999 4.936342139999999
 angles : 60.00000000000001 60.00000000000001 60.00000000000002
 volume : 85.05517878292267
      A : 1.4249992316705467 4.030506479599453 2.4681710700000004
      B : 1.4249992316705467 4.030506479599453 -2.4681710699999995
      C : -2.849998463341093 4.030506479599453 -0.0
    pbc : True True True
PeriodicSite: Y0 (Y) (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Mg1 (Mg) (4.441e-16, 9.069, 1.332e-15) [0.75, 0.75, 0.75]
PeriodicSite: Ni2 (Ni) (0.7078, 6.546, -1.226) [0.3758, 0.8725, 0.3758]
PeriodicSite: Ni3 (Ni) (1.425e-06, 4.544, 2.468e-06) [0.3758, 0.3758, 0.3758]
PeriodicSite: Ni4 (Ni) (-1.416, 6.546, 2.468e-06) [0.3758, 0.3758, 0.8725]
PeriodicSite: Ni5 (Ni) (0.7078, 6.546, 1.226) [0.8725, 0.3758, 0.3758]

In [13]:
SpacegroupAnalyzer(structures[46]).get_space_group_number()

216

In [26]:
len_failed[:3]

[154, 162, 163]

In [9]:
# def permute_conv_lat(lat_mat, space_group):
#     rank = np.argsort(np.linalg.norm(lat_mat,axis=1))
#     perm = lat_perm[space_group, rank[0],rank[1],rank[2]]
#     perm_lat_mat = np.dot(perm, lat_mat)
#     return perm_lat_mat, perm, rank

def permute_conv_lat(lat_mat, space_group):
    rank = np.argsort(-np.linalg.norm(lat_mat,axis=1)) # lat[rank[0]] > lat[rank[1]] > lat[rank[2]] (Descending order)
    perm = lat_perm[space_group, rank[0],rank[1],rank[2]]
    perm_lat_mat = np.dot(perm, lat_mat).dot(perm.T)
    return perm_lat_mat, perm, rank

def project_random_lattice_to_symmetry_prim_conv_permute2(rand_lat_mat, conv_to_prim, space_group):
    rand_conv_lat = np.linalg.inv(conv_to_prim).dot(rand_lat_mat)
    proj_conv_lat = project_lattice(rand_conv_lat, space_group)
    perm_conv_lat, perm, rank = permute_conv_lat(proj_conv_lat.matrix, space_group)
    # proj_prim_lat = np.array(conv_to_prim).dot(perm_conv_lat)
    proj_prim_lat = np.array(conv_to_prim).dot(perm_for_A2[space_group].dot(perm_conv_lat).dot(perm_for_A2[space_group].T))

    return proj_prim_lat, perm_conv_lat, perm, rank

def map_sites_conv_to_prim_lat(conv_sites, conv_species, conv_lat, prim_lat):
    prim_lattice_inv = np.linalg.inv(prim_lat)
    new_fracs = []
    new_species = []
    conv_sites = np.dot(conv_sites, conv_lat)
    for cart_coords, specie in zip(conv_sites, conv_species):
        new_frac = np.dot(cart_coords, prim_lattice_inv)
        # if not any(partial(pbc_diff,new_frac) == partial(f) for f in new_fracs):
        #     new_species.append(site.specie)
        if not any(map(partial(is_periodic_image, new_frac), new_fracs)):
            new_fracs.append(new_frac % 1)
            new_species.append(specie)
        new_fracs = [new_frac % 1 for new_frac in new_fracs]

    # Rhombohedral is not implemented yet
    return new_species, new_fracs


def test_space_group_projection_prim3(structure, print_res=False):
    cif, sym_info, num_sites, formula = process_prim_with_sym_infos(structure)
    space_group = sym_info["spacegroup"]
    prim_plat, conv_plat, perm,rank = project_random_lattice_to_symmetry_prim_conv_permute2(
        initialize_random_lattice().matrix, sym_info["conv_to_prim"], sym_info["spacegroup"]
    )
    # prim_plat = np.dot(perm, prim_plat)
    conv_psites, conv_species = project_sites_prim_with_species(
        initialize_random_atoms(num_sites), sym_info
    )
    # perm_conv_psites = np.dot(perm, conv_psites.T).T
    # conv_psites = perm_conv_psites
    
    # print(psites)
    # conv_species = SpacegroupAnalyzer(structure).get_refined_structure().species
    # print(prim_plat)
    # print("conv_plat",conv_plat)
    proj_conv_structure = Structure(conv_plat, conv_species, conv_psites)
    perm_conv_psite = np.dot(perm, conv_psites.T).T
    perm_conv_plat = perm_for_A2[space_group].dot(conv_plat).dot(perm_for_A2[space_group].T)
    # print(prim_plat)
    prim_species, prim_psites = map_sites_conv_to_prim_lat(
        perm_conv_psite , conv_species, perm_conv_plat, prim_plat
    )
    
    proj_prim_structure = Structure(prim_plat, prim_species, prim_psites)

    res = {
        "proj_conv_structure": proj_conv_structure,
        "proj_prim_structure": proj_prim_structure,
        "structure": structure,
        "sym_info": sym_info,
    }

    try:
        spga = SpacegroupAnalyzer(proj_conv_structure)
    except:
        return {
            **res,
            "correct": False,
            "error": True,
            "correct_species_len": len(structure.species) == len(prim_species.shape[0]),
            "correct_species_len_spga": len(structure.species) == 0,
        }

    spga_prim = spga.get_primitive_standard_structure()
    spg_species_len = len(spga_prim.species)

    if print_res:
        print("Original space group number: ", sym_info["spacegroup"])
        print("Projected space group number: ", spga.get_space_group_number())
        print(
            "Len of species: ",
            len(structure.species),
            len(proj_prim_structure.species),
            spg_species_len,
            len(structure.species) == len(proj_prim_structure.species),
            len(structure.species) == spg_species_len,
        )
        print
    correct = sym_info["spacegroup"] == spga.get_space_group_number()
    return {
        **res,
        "spga_prim": spga_prim,
        "correct": correct,
        "error": False,
        "correct_species_len": len(structure.species) == len(prim_species.shape[0]),
        # "correct_species_len": len(structure.species) == len(proj_prim_structure.species),
        "correct_species_len_spga": len(structure.species) == spg_species_len,
    }



In [27]:
res2 = p_map(test_space_group_projection_prim3, structures, num_cpus=16)

  0%|          | 0/9047 [00:00<?, ?it/s]

/home/holywater2/crystal_gen/mattergen/.venv/lib/python3.10/site-packages/pymatgen/core/periodic_table.py:289: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(


In [31]:
sgn_correct = []
sgn_failed = []
len_correct = []
len_failed = []
spga_len_correct = []
spga_len_failed = []
errors = []

for i in range(len(res2)):
    if res2[i]["correct"]:
        sgn_correct.append(i)
    else:
        sgn_failed.append(i)
    if res2[i]["correct_species_len"]:
        len_correct.append(i)
    else:
        len_failed.append(i)
    if res2[i]["correct_species_len_spga"]:
        spga_len_correct.append(i)
    else:
        spga_len_failed.append(i)
    if res2[i]["error"]:
        errors.append(i)

print(len(sgn_correct), len(len_correct), len(errors), len(spga_len_correct))
print(len(sgn_correct)/len(res2), len(len_correct)/len(res2), len(errors)/len(res2))

9010 9047 29 9018
0.9959102464905494 1.0 0.0032054824803802364


In [29]:
len_failed

[]

In [30]:
spga_len_failed

[45,
 251,
 519,
 1220,
 1451,
 1611,
 1703,
 1989,
 2257,
 2336,
 2793,
 2869,
 3143,
 3899,
 4285,
 4621,
 5416,
 5869,
 6016,
 6062,
 6202,
 6453,
 6759,
 6834,
 6982,
 7088,
 7243,
 7663,
 7750]